In [5]:
import os
import pandas as pd
from PIL import Image
import shutil
import numpy as np
import math

# Directories for images and labels
img_dir = 'train_images_flipped/'
label_dir = 'train_labels_flipped/'

# Directories for output
img_output_dir = 'train_images_flipped_rotated/'
label_output_dir = 'train_labels_flipped_rotated/'

# Make output directories if they do not exist
os.makedirs(img_output_dir, exist_ok=True)
os.makedirs(label_output_dir, exist_ok=True)

# Function to rotate a point around a center


def rotate_point(x, y, cx, cy, angle):
    angle = math.radians(angle)
    dx = x - cx
    dy = y - cy
    new_dx = dx * math.cos(angle) - dy * math.sin(angle)
    new_dy = dx * math.sin(angle) + dy * math.cos(angle)
    return cx + new_dx, cy + new_dy


# Iterate over each file in the image directory
for img_file in os.listdir(img_dir):
    # Open image
    img = Image.open(os.path.join(img_dir, img_file))
    width, height = img.size

    # Read corresponding label
    label_file = os.path.join(label_dir, img_file.rsplit('.', 1)[0] + '.txt')
    labels = pd.read_csv(label_file, header=None)

    # Copy original image and label to new directory
    shutil.copy2(os.path.join(img_dir, img_file),
                 os.path.join(img_output_dir, img_file))
    shutil.copy2(label_file, os.path.join(
        label_output_dir, img_file.rsplit('.', 1)[0] + '.txt'))

    # Generate 3 random rotations
    for _ in range(3):
        angle = np.random.uniform(0, 360)

        # Rotate image
        img_rotated = img.rotate(
            angle, resample=Image.BICUBIC, fillcolor='white')

        # Save rotated image
        img_rotated.save(os.path.join(img_output_dir, img_file.rsplit(
            '.', 1)[0] + f'_rotated{angle:.2f}.jpg'))

        # Rotate bounding boxes
        labels_rotated = labels.copy()
        for i in range(len(labels)):
            for j in range(0, 8, 2):
                x, y = rotate_point(
                    labels.iloc[i, j], labels.iloc[i, j+1], width / 2, height / 2, -angle)
                labels_rotated.iloc[i, j] = x
                labels_rotated.iloc[i, j+1] = y

        # Remove labels with any coordinates outside image bounds
        labels_rotated = labels_rotated[
            ((labels_rotated[0] >= 0) & (labels_rotated[0] <= width) & (labels_rotated[1] >= 0) & (labels_rotated[1] <= height)) &
            ((labels_rotated[2] >= 0) & (labels_rotated[2] <= width) & (labels_rotated[3] >= 0) & (labels_rotated[3] <= height)) &
            ((labels_rotated[4] >= 0) & (labels_rotated[4] <= width) & (labels_rotated[5] >= 0) & (labels_rotated[5] <= height)) &
            ((labels_rotated[6] >= 0) & (labels_rotated[6] <= width) & (
                labels_rotated[7] >= 0) & (labels_rotated[7] <= height))
        ].reset_index(drop=True)

        # Check if labels dataframe is not empty
        if not labels_rotated.empty:
            # Save new labels
            labels_rotated.to_csv(os.path.join(label_output_dir, img_file.rsplit(
                '.', 1)[0] + f'_rotated{angle:.2f}.txt'), header=None, index=None)

print('Augmentation finished.')


Augmentation finished.
